In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
from collections import Counter, defaultdict
from Novelty_KS import knowledge
from Surprise_ES import expectation, dask_pmi, dask_pmi_2
import dask
import dask.dataframe as dd

# Surprise

## First part - to do only one time -- Computing the PMI matrices for each ES

In [2]:
#df_expectation = pd.read_csv('data/expectation_space/cleaned/2016ES_G06F_clean.csv')

In [3]:
#print(len(df_expectation))

In [4]:
#test = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_clean.csv')
#ddf[cols].compute()

In [5]:
#expectation_class = expectation(df_expectation)
#expectation_class.co_occurrence(save=True, path_name ='data/expectation_space/cleaned/2016ES_G06F_PMI.csv')

## With dask

In [2]:
df_expectation = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_clean.csv')
PMI_compute = dask_pmi(df_expectation, partitions= 6)

In [ ]:
#PMI_compute.dask_co_occurrence(save=True, path_name ='data/expectation_space/cleaned/2016ES_G06F_cooccurences.csv')

In [3]:
# Number of part should depend on number of partitions created from dask
ddf1 = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_cooccurences/0.part', sample=10000000)
ddf2 = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_cooccurences/1.part', sample=10000000)
ddf3 = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_cooccurences/2.part', sample=10000000)
ddf4 = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_cooccurences/3.part', sample=10000000)
ddf5 = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_cooccurences/4.part', sample=10000000)
ddf6 = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_cooccurences/5.part', sample=10000000)
ddf_cooc = dd.concat([ddf1, ddf2, ddf3, ddf4, ddf5, ddf6])

In [ ]:
PMI_compute = dask_pmi_2(partitions= 6)
PMI_compute.pmi(ddf_cooc, path_name ='data/expectation_space/cleaned/2016ES_G06F_pmi')

## Score calculation on each dimension

In [2]:
#Import des données nécessaires
#df_eval = pd.read_csv('data/to_eval/cleaned/2016_A61B_patents_toeval_clean.csv')
#df_knowledge = pd.read_csv('data/knowledge_space/cleaned/2016KS_A61B_clean.csv')


In [3]:
df_test = pd.read_csv("data/expectation_space/cleaned/2016ES_G06F_PMI.csv")

In [4]:
df_test.shape

(3314, 162614)

In [ ]:
df_test.head()

In [ ]:
test = dd.read_csv('data/expectation_space/cleaned/2016ES_G06F_PMI.csv')

# Novelty

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from tqdm import tqdm
import collections
import re
from tqdm import tqdm
from collections import Counter, defaultdict
from Novelty_KS import knowledge

In [2]:
def concat_texts(text_list1, text_list2):
    assert len(text_list1) == len(text_list2)
    concat_list = []
    for i in range(len(text_list1)):
        concat_list.append(str(text_list1[i]) + str(text_list2[i]))
    return concat_list

In [3]:
df_toeval = pd.read_csv('data/to_eval/cleaned/2014_A61B_patents_toeval_clean.csv')
print(len(df_toeval))
eval_claims = list(df_toeval['claims'])
eval_back = list(df_toeval['background'])
text_to_eval = eval_claims
print(len(eval_claims))

8383
8383


In [4]:
df_KS1 = pd.read_csv('data/knowledge_space/cleaned/2014KS_A61B_clean.csv')
print(len(df_KS1))

43142


In [5]:
novelty = knowledge(df_KS1)

## newness

In [6]:
Newness_scores = []
for i in tqdm(range(len(text_to_eval))):
    newness = novelty.newness(text_to_eval[i])
    Newness_scores.append(newness)

100%|██████████████████████████████████████████████████████████████████████████████| 8383/8383 [13:20<00:00, 10.47it/s]


In [7]:
print(len(Newness_scores))
print(Newness_scores[0:20])

8383
[0.15184192402798605, 0.5252992362303788, 0.13118684685552312, 0.12620293297197516, 0.14840630455855305, 0.17325325823765894, 0.13598970520575496, 0.2020462007753061, 0.208453690134795, 0.1461401575037168, 0.14620667626301675, 0.15094159022560272, 0.15172766778821187, 0.14650987087255515, 0.16332922882668416, 0.17841972898276834, 0.10882647897357355, 0.15546875340569957, 0.11782604216626835, 0.13123671611424084]


In [8]:
df_toeval['newness_score'] = Newness_scores

In [9]:
df_toeval.to_csv('data/to_eval/results/2014_A61B_RESULTS_CLAIMS.csv', index=False)

## Divergence

In [10]:
avg_ivergence_scores = []
max_ivergence_scores = []
min_ivergence_scores = []
for i in tqdm(range(len(text_to_eval))):
    avg_div, max_div, min_div = novelty.divergence(text_to_eval[i])
    avg_ivergence_scores.append(avg_div)
    max_ivergence_scores.append(max_div)
    min_ivergence_scores.append(min_div)

100%|██████████████████████████████████████████████████████████████████████████████| 8383/8383 [19:22<00:00,  7.21it/s]


In [11]:
#print(len(Divergence_scores))
print(avg_ivergence_scores[0:20], max(avg_ivergence_scores), min(avg_ivergence_scores))

[0.9632053560681801, 0.9522224936075585, 0.9518078130155673, 0.9503416267184038, 0.9693876924211458, 0.9743101943892565, 0.9656185808425161, 0.9697740993208025, 0.9571707328039489, 0.9645014688586205, 0.9785935956556441, 0.9660613059858019, 0.950933732025813, 0.9633728285874607, 0.9439022022680336, 0.9839152023809589, 0.9437991310814493, 0.9518759085328331, 0.9538677594076584, 0.9428945022924083] 0.9918266896954281 0.9093627300119493


In [12]:
df_results = pd.read_csv('data/to_eval/results/2014_A61B_RESULTS_CLAIMS.csv')
df_results['average divergence_score'] = avg_ivergence_scores
df_results['max divergence_score'] = max_ivergence_scores
df_results['min divergence_score'] = min_ivergence_scores

In [13]:
df_results.to_csv('data/to_eval/results/2014_A61B_RESULTS_CLAIMS.csv', index=False)

## uniqueness

In [14]:
unique_scores = []
delta_uniq = []
for i in tqdm(range(len(text_to_eval))):
    uniqueness, delta = novelty.uniqueness(text_to_eval[i])
    unique_scores.append(uniqueness)
    delta_uniq.append(delta)

100%|██████████████████████████████████████████████████████████████████████████████| 8383/8383 [26:19<00:00,  5.31it/s]


In [15]:
print(unique_scores[0:20], max(unique_scores), min(unique_scores))
print(delta_uniq[0:20], max(delta_uniq), min(delta_uniq))

[0.8235878854518941, 0.7709304934124592, 0.768942305125305, 0.7619126671844315, 0.853229129729058, 0.876830091398118, 0.8351581043704229, 0.8550817596490723, 0.7946548523583583, 0.8298021050670681, 0.897366881379336, 0.8372807524067885, 0.7647515192061756, 0.8243908333295717, 0.7310388126672479, 0.9228813529134483, 0.7305446373191185, 0.7692687894422721, 0.7788187289754154, 0.7262073898150649] 0.9608130206026967 0.5654391306221648
[-0.16795045875209003, -0.2206078507915249, -0.2225960390786791, -0.2296256770195526, -0.13830921447492617, -0.11470825280586616, -0.1563802398335612, -0.13645658455491183, -0.1968834918456258, -0.16173623913691604, -0.09417146282464817, -0.15425759179719567, -0.22678682499780856, -0.16714751087441237, -0.2604995315367362, -0.06865699129053582, -0.2609937068848657, -0.22226955476171206, -0.21271961522856875, -0.2653309543889192] -0.030725323601287435 -0.42609921358181935


In [16]:
df_results = pd.read_csv('data/to_eval/results/2014_A61B_RESULTS_CLAIMS.csv')
df_results['uniqueness'] = unique_scores
df_results['delta uniqueness'] = delta_uniq

In [17]:
df_results.to_csv('data/to_eval/results/2014_A61B_RESULTS_CLAIMS.csv', index=False)